# Test SSO Login

The instance has all the permissions required, now run the login command for the `Jupyter-IR-ViewOnly` role and the loggin account.

In [ ]:
import boto3, os
from jupyterirtools import sso

sso.login("Jupyter-IR-ViewOnly")

In [ ]:
sso.print_permissions()

In [ ]:
boto3.setup_default_session()
sts = boto3.client('sts')
identity = sts.get_caller_identity()
print(f"UserId: {identity['UserId']}\nAccount: {identity['Account']}\n")

In [ ]:
session = sso.get_session("Jupyter-IR-AdministratorAccess",os.environ['MANAGEMENT_ACCOUNT'])
sts = session.client('sts')
identity = sts.get_caller_identity()
print(f"UserId: {identity['UserId']}\nAccount: {identity['Account']}\n")

In [ ]:
! aws sts get-caller-identity


In [ ]:
sso.logout()

In [ ]:
boto3.setup_default_session()
sts = boto3.client('sts')
try:
    sts.get_caller_identity()
    print("The logout FAILED")
except:
    print("The logout was successful")